## A 3d Demo

The notebook shows how to extract SN-Graph out of a 3D volume.

In [ ]:
import trimesh
import numpy as np
from sn_graph import create_sn_graph, draw_sn_graph

In [ ]:
mesh = trimesh.load("../assets/cup_0085.off")
print("Displaying original mesh...")
mesh.show()

In [ ]:
# local voxelise allows to fill the inside of the mesh with positive values which is needed for sdf computation
voxels = trimesh.voxel.creation.local_voxelize(
    mesh, point=(0, 0, 0), radius=300, pitch=0.5, fill=True
)
print("Displaying voxelized mesh...")
voxels.show()

In [ ]:
binary_volume = np.array(voxels.matrix.astype(int))

graph = create_sn_graph(binary_volume, return_sdf=True, minimal_sphere_radius=2)
graph_volume = draw_sn_graph(*graph, draw_circles=True)

print(f"No vertices: {len(graph[0])}.")
print(f"No edges: {len(graph[1])}.")

In [ ]:
graph_voxels = trimesh.voxel.VoxelGrid(graph_volume).as_boxes()
print("Displaying voxelized graph...")
graph_voxels.show()

In [ ]:
def visualize_3d_graph(
    vertices: list, edges: list, sdf_array: np.ndarray
) -> trimesh.scene.scene.Scene:
    """
    Visualize a graph with vertices, edges, and SDF spheres

    Parameters:
    -----------
    vertices : list of coordinate tuples [(x1,y1,z1), (x2,y2,z2), ...]
    edges : list of tuples of coordinates for start and end of edges [((x1,y1,z1), (x2,y2,z2)), ...]
    sdf_array : array that can be queried at vertex coordinates to get radius
    """
    # Create a scene
    scene = trimesh.Scene()
    # Add edges directly as line segments - blue and thick
    for start_coord, end_coord in edges:
        # Create a line segment between start and end
        line = trimesh.creation.cylinder(
            radius=0.03,  # Thick lines
            segment=[start_coord, end_coord],
        )
        line.visual.face_colors = [0, 0, 255, 255]  # Blue
        scene.add_geometry(line)

    # Add spheres and vertex points for each vertex
    for v in vertices:
        # Get radius from SDF array
        radius = sdf_array[tuple(v)]

        # Create a smooth sphere based on SDF value
        sphere = trimesh.creation.icosphere(radius=radius, subdivisions=2)
        sphere.visual.face_colors = [255, 0, 0, 255]  # Red
        sphere.apply_translation(v)
        scene.add_geometry(sphere)

        # Add small vertex points
        point = trimesh.creation.icosphere(radius=0.02)
        point.visual.face_colors = [0, 0, 0, 255]  # Black
        point.apply_translation(v)
        scene.add_geometry(point)

    return scene


scene = visualize_3d_graph(*graph)
print("Displaying graph (as a 3D mesh)...")
scene.show()